In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os

def power_law(x, a, b):
    return a * np.power(x, b)

def fit_power_law(x, y):
    # 只拟合非零且非nan的点
    mask = (~np.isnan(x)) & (~np.isnan(y)) & (x > 0) & (y > 0)
    x_fit = x[mask]
    y_fit = y[mask]
    if len(x_fit) < 2:
        return np.nan, np.nan
    try:
        popt, _ = curve_fit(power_law, x_fit, y_fit, maxfev=10000)
        return popt[0], popt[1]
    except Exception as e:
        return np.nan, np.nan

def plot_power_law_fit(x, y, a, b, label, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(7,5))
    ax.scatter(x, y, label='points', color='blue', alpha=0.6, s=30)
    if not (np.isnan(a) or np.isnan(b)):
        x_fit = np.linspace(np.nanmin(x), np.nanmax(x), 200)
        y_fit = power_law(x_fit, a, b)
        ax.plot(x_fit, y_fit, color='red', lw=2.5, label=f'power equation: y={a:.2f}x^{b:.2f}')
    ax.set_xlabel('Row Sum')
    ax.set_ylabel(label)
    ax.set_title(f'{label} 幂函数拟合')
    ax.legend()
    ax.grid(True)
    return ax

if __name__ == "__main__":
    # 设置输入和输出目录
    input_dir = r"D:\Work\work\通量塔\siji\新建文件夹"
    output_dir = r"D:\Work\work\通量塔\0827\幂函数拟合"
    os.makedirs(output_dir, exist_ok=True)
    files = [
        "S1_times.csv",
        "S2_times.csv",
        "S3_times.csv",
        "S4_times.csv"
    ]
    for file_name in files:
        file_path = os.path.join(input_dir, file_name)
        print(f"处理文件: {file_name}")
        try:
            # 读取数据
            header = pd.read_csv(file_path, nrows=0)
            df = pd.read_csv(file_path, skiprows=1, nrows=1200, names=header.columns)
            # 只保留数值型列
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            if len(numeric_cols) == 0:
                print(f"文件 {file_name} 没有数值型列，跳过。")
                continue
            df_numeric = df[numeric_cols]
            # 归一化
            df_norm = (df_numeric - df_numeric.min()) / (df_numeric.max() - df_numeric.min())
            row_sums = df_norm.sum(axis=1)
            df_new = pd.DataFrame({'Row Sum': row_sums})
            df_new = pd.concat([df_new, df_norm], axis=1)
            df_new = df_new.loc[row_sums.sort_values().index]

            #保存归一化后的数据
            norm_output_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_normalized.csv")
            df_new.to_csv(norm_output_path, index=False)    
            print(f"归一化数据已保存到: {norm_output_path}")
    
            # 为每一列进行幂函数拟合并绘图
            fig, axes = plt.subplots(2, 2, figsize=(15, 15))
            axes = axes.ravel()
                
            for i, col in enumerate(df_norm.columns):
                    if i >= 4:  # 限制只显示前4个图
                        break
                    a, b = fit_power_law(df_new['Row Sum'], df_new[col])
                    plot_power_law_fit(df_new['Row Sum'], df_new[col], a, b, col, ax=axes[i])
                
            plt.tight_layout()
            fig_output_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_power_law_fits.png")
            plt.savefig(fig_output_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"幂函数拟合图已保存到: {fig_output_path}")
                
        except Exception as e:
                print(f"处理文件 {file_name} 时发生错误: {str(e)}")

处理文件: S1_times.csv
归一化数据已保存到: D:\Work\work\通量塔\0827\幂函数拟合\S1_times_normalized.csv


C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 24130 (\N{CJK UNIFIED IDEOGRAPH-5E42}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 20989 (\N{CJK UNIFIED IDEOGRAPH-51FD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 25311 (\N{CJK UNIFIED IDEOGRAPH-62DF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 21512 (\N{CJK UNIFIED IDEOGRAPH-5408}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:86: UserWarning: Glyph 24130 (\

幂函数拟合图已保存到: D:\Work\work\通量塔\0827\幂函数拟合\S1_times_power_law_fits.png
处理文件: S2_times.csv
归一化数据已保存到: D:\Work\work\通量塔\0827\幂函数拟合\S2_times_normalized.csv


C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 24130 (\N{CJK UNIFIED IDEOGRAPH-5E42}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 20989 (\N{CJK UNIFIED IDEOGRAPH-51FD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 25311 (\N{CJK UNIFIED IDEOGRAPH-62DF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 21512 (\N{CJK UNIFIED IDEOGRAPH-5408}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:86: UserWarning: Glyph 24130 (\

幂函数拟合图已保存到: D:\Work\work\通量塔\0827\幂函数拟合\S2_times_power_law_fits.png
处理文件: S3_times.csv
归一化数据已保存到: D:\Work\work\通量塔\0827\幂函数拟合\S3_times_normalized.csv


C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 24130 (\N{CJK UNIFIED IDEOGRAPH-5E42}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 20989 (\N{CJK UNIFIED IDEOGRAPH-51FD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 25311 (\N{CJK UNIFIED IDEOGRAPH-62DF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 21512 (\N{CJK UNIFIED IDEOGRAPH-5408}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:86: UserWarning: Glyph 24130 (\

幂函数拟合图已保存到: D:\Work\work\通量塔\0827\幂函数拟合\S3_times_power_law_fits.png
处理文件: S4_times.csv
归一化数据已保存到: D:\Work\work\通量塔\0827\幂函数拟合\S4_times_normalized.csv


C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 24130 (\N{CJK UNIFIED IDEOGRAPH-5E42}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 20989 (\N{CJK UNIFIED IDEOGRAPH-51FD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 25311 (\N{CJK UNIFIED IDEOGRAPH-62DF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:84: UserWarning: Glyph 21512 (\N{CJK UNIFIED IDEOGRAPH-5408}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\18804\AppData\Local\Temp\ipykernel_14044\1518129398.py:86: UserWarning: Glyph 24130 (\

幂函数拟合图已保存到: D:\Work\work\通量塔\0827\幂函数拟合\S4_times_power_law_fits.png
